# Imports

In [ ]:
#before runing the code pip install the following: 
#pip install tensorflow
#pip install tensorflow_text
#pip install tensorflow_hub

from pymongo import MongoClient
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
import openai
import nest_asyncio
nest_asyncio.apply()
import tensorflow_hub as hub
import datetime
from datetime import timedelta
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

from marktrix.helpers import Helper
from marktrix.calendar import Calendar, global_calendar
from marktrix.user_data import UserData
from marktrix.website_info import WebsiteInfo

pd.set_option('display.max_rows', None)

# Load your API key from an environment variable or secret management service
openai.api_key = "sk-9GD2atPAt9M1Od4YaPWIew7J3BNSlWKiO1d0Z7z3"

#load universal sentence encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

embed = hub.load(module_url)


# Globals

In [ ]:
load_dotenv()

In [ ]:
mongo_url = os.environ.get("MARKTRIX_MONGO")
client = MongoClient(mongo_url)
db = client['marktrix-production']
# db = client['marktrix-staging']

# Predict best calendar events from USA market Calendar

In [ ]:
#a function that return max events embed similarity to user and target audience 
def predict_best_calendar_events(user_info, keyword, start_date, n_best):
    #calculate events dates
    events_start_date = start_date + timedelta(days = 7)
    events_end_date = start_date + timedelta(days = 21)
    keyword = str(keyword)
    n_best = int(n_best)
    
    #user and website info
    company_name = Helper.get_object_field(user_info, 'CompanyName')
    company_industry = Helper.get_object_field(user_info, 'OwnIndustry')
    site_desc = Helper.get_object_field(user_info, 'SiteDescription')
    sol_desc = Helper.get_object_field(user_info, 'SolutionDescription')
    num_one_benefit = Helper.get_object_field(user_info, 'NumberOneBenefit')
    company_size = Helper.get_object_field(user_info, 'OwnCompanySize') 
    main_product_categories = str(Helper.get_object_field(user_info, 'MainProductCategories'))
    #business_model = Helper.get_object_field(user_info, 'BusinessModel')  # for Tone
    #business_model = Helper.business_model_desc(business_model)
    #how_customer_deal_with_needs = Helper.get_object_field(user_info, 'HowCustomerDealWithNeeds')
    #sol_change = Helper.get_object_field(user_info, 'WhatYourSolutionChange')
    #why_sol_important = Helper.get_object_field(user_info, 'WhySolutionImportant')
    company_text = keyword +' '+ company_name + ' ' + company_industry + ' ' + site_desc + ' ' + sol_desc + \
                    ' ' + num_one_benefit +  ' ' + company_size  + ' ' + main_product_categories
       
    
    #create embeddings from company's text and target customers
    company_text = tf.convert_to_tensor(company_text.astype(str), dtype=tf.string)
    company_embed = embed(company_text)
    
    # target customer info
    target_customer_description = Helper.get_object_field(user_info, 'TargetCustomerDescription')
    positive_trends = Helper.get_object_field(user_info, 'PositiveTrends')
    
    target_customer_ages = Helper.get_object_field(user_info, 'TargetCustomerAges')
    if target_customer_ages.iloc[0] != '' and type(target_customer_ages.iloc[0]) == list:
        target_customer_ages = [Helper.customer_age_to_text(age) for age in target_customer_ages]
        target_customer_ages = Helper.list_to_text(target_customer_ages)

    target_customer_gender = Helper.get_object_field(user_info, 'TargetCustomerGender')
    if target_customer_gender.iloc[0] != '' and type(target_customer_gender.iloc[0]) == list:
        target_customer_gender = Helper.list_to_text(target_customer_gender)

    target_customer_geo = Helper.get_object_field(user_info, 'TargetCustomerGeography')
    if target_customer_geo.iloc[0] != '' and type(target_customer_geo.iloc[0]) == list:
        target_customer_geo = Helper.list_to_text(target_customer_geo)

     # decide if to use old or new
    if target_customer_ages.iloc[0] or target_customer_gender.iloc[0] or target_customer_geo.iloc[0]:
         target_customer_description = target_customer_ages + ' '  + target_customer_gender+ ' ' + target_customer_geo
    
    
    target_customer_and_trends_text = target_customer_description + ' ' + positive_trends 

    
    #If target customer text and trends is empty use Company_text
    if  target_customer_and_trends_text is not '':
        target_customer_and_trends_text= tf.convert_to_tensor(target_customer_and_trends_text.astype(str), dtype=tf.string)
        target_customers_embed = embed(target_customer_and_trends_text)
       
    else:
        target_customers_embed = embed(company_text)

    
    #get usa_market_calendar
    usa_market_calendar = list(db.usa_market_calendar.find())
    usa_market_calendar = pd.DataFrame(usa_market_calendar)
    usa_market_calendar ['date'] = usa_market_calendar['date'].dt.date
    mask = (usa_market_calendar['date'] > events_start_date) & (usa_market_calendar['date'] <= events_end_date)
    conc_usa_market_events = usa_market_calendar.loc[mask]
    conc_usa_market_events = conc_usa_market_events[['name','desc', 'date']] 
    conc_usa_market_events_name = conc_usa_market_events['name'] 
    conc_usa_market_events_desc = conc_usa_market_events['desc'] 
    conc_usa_market_events['text'] = conc_usa_market_events_name + ' ' + conc_usa_market_events_desc

    #create embeddings from concurrent events texts
    conc_usa_market_events['text'] = tf.convert_to_tensor(conc_usa_market_events['text'].astype(str), dtype=tf.string)
    conc_usa_market_events_embed = embed(conc_usa_market_events['text'])
        
    #calculate target customer embed correlation with concurrent_events_embed 
    target_event_cos_sim = tf.keras.losses.cosine_similarity(conc_usa_market_events_embed, target_customers_embed)

    #calculate company embed correlation with concurrent_events_embed
    company_event_cos_sim = tf.keras.losses.cosine_similarity(conc_usa_market_events_embed, company_embed)


    #calculates a similarity score based on combination of company and target customers similarity scores and contact with the best usa market events
    similarity_score = target_event_cos_sim*0.5 + company_event_cos_sim*0.5
    conc_usa_market_events['similarity_score'] =  similarity_score
    conc_usa_market_events['similarity_score'] = conc_usa_market_events['similarity_score'].astype(float)
    best = max(n_best,len(conc_usa_market_events['similarity_score']))
    best = int(best)
    df = conc_usa_market_events.nlargest(best, 'similarity_score')
    df = df.sort_values(by = ['similarity_score'], ascending=False)
    df = df[['name','desc', 'date','similarity_score']]            

        
    return df 
    #with dates (can be index), pred, event. - sorted top pred first, event is event object as in calendar (day or month)

# Test the function

In [ ]:
user_data = UserData()
website_info = WebsiteInfo()
users_info = {}

In [ ]:
#get user answers
cursor = db.useranswers.find()
users= [item for item in cursor]
data = list(db.useranswers.find())
data = pd.DataFrame(data)
data.head(10)
# create a pivot table where each user has one row and placeholders as columns
pivot_table = data.pivot_table(index = 'user', columns = 'placeholder', values = 'originalAnswer', aggfunc = 'first')
#reset index so the user will be presented as column and not index
df = pivot_table.reset_index(level = 'user', inplace = False)
website_info = WebsiteInfo()
df['SiteDescription'] = df['user'].apply(lambda x: website_info.get_user_website_info('production', x))
df['SiteDescription'] = df['SiteDescription'].astype(str)
user = 'zuzusandals@gmail.com'
user_info = df.loc[df['user'] == user]
user_info.to_dict()
global_calendar = global_calendar
keyword = 'Barefoot Sandals'
n_best = 5
start_date  = datetime.date(2023, 2, 12)




In [ ]:
events_df = predict_best_calendar_events(user_info, keyword, start_date, n_best)

In [ ]:
# function result
events_df